In [1]:
import pandas as pd

Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [9]:
yelpparsed=pd.read_csv('../data/yelpdata.csv')
foursquareparsed=pd.read_csv('../data/foursqdata.csv')
citybikedata=pd.read_csv('../data/data.csv')

In [7]:
mergeddf=pd.merge(yelpparsed,foursquareparsed,on=['name','location.address'],how='inner',suffixes=('yelp','4sq'))
mergeddf.head()
mergeddf = mergeddf[['stationyelp','name','location.address','ratingyelp','rating4sq']]

In [8]:
mergeddf['ratingyelp']=mergeddf[['ratingyelp']]*2
mergeddf.head()

,stationyelp,name,location.address,ratingyelp,rating4sq
0,East End Library,Kahlo’s Taqueria and Bar,718 N 23rd St,8.2,7.8
1,East End Library,Union Market,2306 Jefferson Ave,8.6,9.4
2,East End Library,Patrick Henry's Pub & Grille,2300 E Broad St,7.8,8.6
3,Sydney Park,Cobra Cabana,901 W Marshall St,8.6,7.9
4,Sydney Park,Home Team Grill,1630 W Main St,6.8,6.5


Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

Look at the data before and after the join to validate your data.